# Classical Neural Network on the Snacks Dataset

In [1]:
import numpy as np
import keras
from keras.models import Sequential
from keras.layers import *
from keras import optimizers

Using TensorFlow backend.


## Building the model

In [2]:
image_width = 32
image_height = 32
num_classes = 20

In [3]:
model = Sequential()
model.add(Flatten(input_shape=(image_height, image_width, 3)))
model.add(Dense(500, activation="relu"))  # this line is new
model.add(Dense(num_classes))
model.add(Activation("softmax"))

Instructions for updating:
Colocations handled automatically by placer.


In [4]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten_1 (Flatten)          (None, 3072)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 500)               1536500   
_________________________________________________________________
dense_2 (Dense)              (None, 20)                10020     
_________________________________________________________________
activation_1 (Activation)    (None, 20)                0         
Total params: 1,546,520
Trainable params: 1,546,520
Non-trainable params: 0
_________________________________________________________________


In [5]:
model.compile(loss="categorical_crossentropy",
              optimizer=optimizers.Adam(lr=1e-3),
              metrics=["accuracy"])

## Loading the data

In [6]:
images_dir = "snacks/"
train_data_dir = images_dir + "train/"
val_data_dir = images_dir + "val/"
test_data_dir = images_dir + "test/"

In [7]:
def normalize_pixels(image):
    return image / 127.5 - 1

In [8]:
from keras.preprocessing.image import ImageDataGenerator
datagen = ImageDataGenerator(preprocessing_function=normalize_pixels)

In [9]:
batch_size = 64

train_generator = datagen.flow_from_directory(
                    train_data_dir,
                    target_size=(image_width, image_height),
                    batch_size=batch_size,
                    class_mode="categorical",
                    shuffle=True)

val_generator = datagen.flow_from_directory(
                    val_data_dir,
                    target_size=(image_width, image_height),
                    batch_size=batch_size,
                    class_mode="categorical",
                    shuffle=False)

test_generator = datagen.flow_from_directory(
                    test_data_dir,
                    target_size=(image_width, image_height),
                    batch_size=batch_size,
                    class_mode="categorical",
                    shuffle=False)

Found 4838 images belonging to 20 classes.
Found 955 images belonging to 20 classes.
Found 952 images belonging to 20 classes.


In [10]:
index2class = {v:k for k,v in train_generator.class_indices.items()}

## Evaluating the untrained model on the entire dataset

In [11]:
model.evaluate_generator(test_generator, steps=len(test_generator))

[3.352704174378339, 0.04831932773109244]

In [12]:
model.evaluate_generator(train_generator, steps=len(train_generator))

[3.2993269887232692, 0.043819760231500623]

In [13]:
model.evaluate_generator(val_generator, steps=len(val_generator))

[3.312240543665062, 0.0387434554973822]

## Training

In [12]:
import warnings
warnings.filterwarnings("ignore")

In [13]:
model.fit_generator(train_generator, 
                    steps_per_epoch=len(train_generator),
                    validation_data=val_generator,
                    validation_steps=len(val_generator),
                    epochs=3,
                    workers=4)

Instructions for updating:
Use tf.cast instead.
Epoch 1/3
76/76 [==============================] - 47s 621ms/step - loss: 3.2180 - acc: 0.1235 - val_loss: 3.1232 - val_acc: 0.1435
Epoch 2/3
76/76 [==============================] - 5s 62ms/step - loss: 2.4350 - acc: 0.3077 - val_loss: 3.1800 - val_acc: 0.1508
Epoch 3/3
76/76 [==============================] - 5s 66ms/step - loss: 1.9486 - acc: 0.4554 - val_loss: 3.2983 - val_acc: 0.1424


In [14]:
model.evaluate_generator(test_generator, steps=len(test_generator))

[3.274118868242793, 0.13550420155545242]